<a href="https://colab.research.google.com/github/Apekshaa2908/Social-Media-Hashtag-Trend-Analyzer-Application/blob/main/Social_Media_Hashtag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile app.py
import streamlit as st
import requests
import uuid
import json

# AWS Configuration
aws_access_key_id = 'aws_access_key_id'
aws_secret_access_key = 'aws_secret_access_key'
region_name = 'region_name'
lambda_url = 'lambda_url'  # Replace with your API Gateway endpoint

# Streamlit UI components
st.title('Post Composer')

# Input field for text content
post_content = st.text_area('Compose your post:', height=100)

# Input field for hashtags
hashtags = st.text_input('Add hashtags (comma-separated):')

# Button to submit the post
if st.button('Publish'):
    if post_content and hashtags:
        # Process the post and hashtags
        post_data = {
            'post_id': str(uuid.uuid4()),
            'content': post_content,
            'hashtags': [tag.strip() for tag in hashtags.split(',') if tag.strip()]
        }

        # Send the post data to AWS Lambda
        try:
            response = requests.post(lambda_url, json=post_data)
            if response.status_code == 200:
                st.success('Post published successfully!')
                st.write(response.json())  # Display response body
            else:
                st.error(f'Failed to publish post. Status code: {response.status_code}')
                st.write(f'Response: {response.text}')
        except Exception as e:
            st.error(f'Error publishing post: {e}')
    else:
        st.error('Post content or hashtags are missing.')

st.title('Trending Hashtags')

# Button to show trending hashtags
if st.button('Show Trending Hashtags'):
    # Query DynamoDB table to fetch trending hashtags
    try:
        response = table.scan()
        hashtags_count = {}
        for item in response['Items']:
            for hashtag in item.get('hashtags', []):
                hashtags_count[hashtag] = hashtags_count.get(hashtag, 0) + 1

        # Sort hashtags by count in descending order
        sorted_hashtags = sorted(hashtags_count.items(), key=lambda x: x[1], reverse=True)

        # Display top 3 trending hashtags
        st.write("Top 3 Trending Hashtags:")
        for hashtag, count in sorted_hashtags[:3]:
            st.write(f"{hashtag}: {count} posts")
    except Exception as e:
        st.error(f'Error fetching trending hashtags: {e}')


In [ ]:
! wget -q -O - ipv4.icanhazip.com
! streamlit run app.py & npx localtunnel --port 8501